In [58]:
import os
import time
from datetime import datetime
import pytz
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials

# Set the timezone to CST
cst_timezone = pytz.timezone('America/Chicago')

# Record the current system time in CST as the capture time
capture_time = datetime.now(cst_timezone)

# Set the environment variables using os.environ
os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY'] = 'ada90a982a7a4bebbc38fa123565d7d6'
os.environ['COMPUTER_VISION_ENDPOINT'] = 'https://vehiclerg.cognitiveservices.azure.com/'

# Read the environment variables using os.environ
subscription_key = os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY']
endpoint = os.environ['COMPUTER_VISION_ENDPOINT']

# subscription_key = '96a5175a532f4c7e876c576e00cbc3fe'
# endpoint = 'https://vehiclenp.cognitiveservices.azure.com/'
credentials = CognitiveServicesCredentials(subscription_key)
client = ComputerVisionClient(endpoint, credentials)
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

# Load the image to extract text from
image_url = "https://licenseplateimagesdover.blob.core.windows.net/licenseplateimages/image1.jpeg"


# image_url = r"C:\\Users\\Administrator\\Desktop\\DoverHackathon\\LICENSEPLATES\\image1.jpeg"


# Call the OCR method on the image
read_response = computervision_client.read(image_url, raw=True)

# Get the operation location (URL with an ID at the end) from the response
read_operation_location = read_response.headers["Operation-Location"]

# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]


# Initialize variables to store state and license plate
state = None
license_plate = None

state_abbreviations = {
    'alabama': 'AL', 'alaska': 'AK', 'americansamoa': 'AS', 'arizona': 'AZ', 'arkansas': 'AR', 
    'california': 'CA', 'colorado': 'CO', 'connecticut': 'CT', 'delaware': 'DE', 
    'districtofcolumbia': 'DC', 'florida': 'FL', 'georgia': 'GA', 'guam': 'GU', 'hawaii': 'HI', 
    'idaho': 'ID', 'illinois': 'IL', 'indiana': 'IN', 'iowa': 'IA', 'kansas': 'KS', 
    'kentucky': 'KY', 'louisiana': 'LA', 'maine': 'ME', 'maryland': 'MD', 
    'massachusetts': 'MA', 'michigan': 'MI', 'minnesota': 'MN', 'mississippi': 'MS', 
    'missouri': 'MO', 'montana': 'MT', 'nebraska': 'NE', 'nevada': 'NV', 
    'newhampshire': 'NH', 'newjersey': 'NJ', 'newmexico': 'NM', 'newyork': 'NY', 
    'northcarolina': 'NC', 'northdakota': 'ND', 'northernmarianaislands': 'MP', 
    'ohio': 'OH', 'oklahoma': 'OK', 'oregon': 'OR', 'pennsylvania': 'PA', 'puertorico': 'PR', 
    'rhodeisland': 'RI', 'southcarolina': 'SC', 'southdakota': 'SD', 'tennessee': 'TN', 
    'texas': 'TX', 'utah': 'UT', 'vermont': 'VT', 'usvirginislands': 'VI', 'virginia': 'VA', 
    'washington': 'WA', 'westvirginia': 'WV', 'wisconsin': 'WI', 'wyoming': 'WY'
}


# # Convert each state name to lowercase and remove spaces
# clean_states = [state.lower().replace(" ", "") for state in states]

# print(clean_states)

# Call the "GET" API and wait for it to retrieve the results
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Process the detected text
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            text = line.text.lower()  # Convert to lower case for easier matching

            # Check if the line contains a license plate number
            if any(char.isdigit() for char in text) and any(char.isalpha() for char in text):
                license_plate = text
                # print("License plate number:", license_plate)

            # Check for state name
            for state_name in state_abbreviations:
                if state_name in text:
                    state = state_name  # Capture the matched state name
                    short_state = state_abbreviations.get(state, None)  # Get the abbreviation
                    # print("State:", state)
                    # print("Short State:", short_state)
                    break  # Exit the loop if a state name is found

            # Exit the loop if both state and license plate number are found
            if state and license_plate:
                break

cleaned_license_plate = ''.join(filter(str.isalnum, license_plate))

# Print 
print("Capture Time:", capture_time.strftime("%Y-%m-%d %H:%M:%S %Z"))     
print("State:", state)
print("Short State:", short_state)
print("License Plate:", cleaned_license_plate)

Capture Time: 2024-02-03 23:43:19 CST
State: texas
Short State: TX
License Plate: lvl0106


In [70]:
import requests
import xml.etree.ElementTree as ET

def get_vehicle_data(registration_number, state, username):
    url = 'https://www.vehicleregistrationapi.com/api/reg.asmx/CheckUSA'
    params = {
        'RegistrationNumber': registration_number,
        'State': state,
        'username': username
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.text
    else:
        return None

def parse_xml(xml_data):
    # Namespace dictionary to handle the default namespace in XML
    ns = {'ns': 'http://regcheck.org.uk'}
    root = ET.fromstring(xml_data)

    # Extracting specific fields from the XML
    description = root.find('.//ns:Description', ns).text if root.find('.//ns:Description', ns) is not None else 'N/A'
    registration_year = root.find('.//ns:RegistrationYear', ns).text if root.find('.//ns:RegistrationYear', ns) is not None else 'N/A'
    car_make = root.find('.//ns:CarMake/ns:CurrentTextValue', ns).text if root.find('.//ns:CarMake/ns:CurrentTextValue', ns) is not None else 'N/A'
    car_model = root.find('.//ns:CarModel', ns).text if root.find('.//ns:CarModel', ns) is not None else 'N/A'
    body_style = root.find('.//ns:BodyStyle/ns:CurrentTextValue', ns).text if root.find('.//ns:BodyStyle/ns:CurrentTextValue', ns) is not None else 'N/A'
    engine_size = root.find('.//ns:EngineSize/ns:CurrentTextValue', ns).text if root.find('.//ns:EngineSize/ns:CurrentTextValue', ns) is not None else 'N/A'

    return {
        'Description': description,
        'RegistrationYear': registration_year,
        'CarMake': car_make,
        'CarModel': car_model,
        'BodyStyle': body_style,
        'EngineSize': engine_size
    }



In [71]:
# Example usage
registration_number = cleaned_license_plate
state = short_state
username = 'shrawani'

xml_response = get_vehicle_data(registration_number, state, username)

# Initialize variables for vehicle info
description = None
registration_year = None
car_make = None
car_model = None
body_style = None
engine_size = None

if xml_response:
    vehicle_info = parse_xml(xml_response)
    if vehicle_info:
        # Storing each variable
        description = vehicle_info.get('Description', None)
        registration_year = vehicle_info.get('RegistrationYear', None)
        car_make = vehicle_info.get('CarMake', None)
        car_model = vehicle_info.get('CarModel', None)
        body_style = vehicle_info.get('BodyStyle', None)
        engine_size = vehicle_info.get('EngineSize', None)
    else:
        print("No vehicle data found in the XML.")
else:
    print("Failed to retrieve data")



In [73]:
from dotenv import load_dotenv
load_dotenv()

from azure.data.tables import TableServiceClient
import os

# AZURE_STORAGE_CONNECTION_STRING = 'DefaultEndpointsProtocol=https;AccountName=customerprofile;AccountKey=cyVAvDM1rAwKv80HDN2xT+on06xVEXoTd9S5ef3WVg8W0iarZFWhGl4z1yQwSl4CL6amZMggc7Vm+AStx0dhag==;EndpointSuffix=core.windows.net'
# Set the connection string
connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

# Create a TableServiceClient using the connection string
table_service = TableServiceClient.from_connection_string(conn_str=connection_string)

# Name of the table
table_name = 'VehicleData'

# Create a new table if it doesn't exist
table_client = table_service.create_table_if_not_exists(table_name=table_name)

# Define your vehicle data
vehicle_data = {
    "PartitionKey": state,  # or any other category suitable for partitioning
    "RowKey": str(capture_time),  # Unique row key, e.g., a timestamp
    "CaptureTime": capture_time,
    "State": state,
    "ShortState": short_state,
    "LicensePlate": license_plate,
    "Description": description,
    "RegistrationYear": registration_year,
    "CarMake": car_make,
    "CarModel": car_model,
    "BodyStyle": body_style,
    "EngineSize": engine_size
}

# Insert the data into the table
table_client.create_entity(entity=vehicle_data)

print("Data inserted successfully into Azure Table Storage.")


Data inserted successfully into Azure Table Storage.
